<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/All.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

In [ ]:
!pip install tensorflow

In [ ]:
!pip install qdrant-client

#detect crop and segment

In [ ]:
import os
from PIL import Image
import cv2
import numpy as np
from ultralytics import YOLO
from qdrant_client import QdrantClient
from transformers import CLIPProcessor, CLIPModel
import torch
import matplotlib.pyplot as plt

# Load your YOLOv8 segmentation model
model = YOLO('/content/drive/MyDrive/drug/best_segment.pt')

# Define the input directory
input_dir = '/content/data 50 class add_augment'

# Connect to Qdrant
qdrant_client = QdrantClient(
    url="https://8366dca9-9b40-481c-9a7c-102b62b118c2.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="Ft3t6L99uoTsJW8IOk5VH2byKU-BQYqiieuxXFBDp99wo75od0ddAw",
)
collection_name = "vector_drug"

# Load CLIPModel and processor for embedding generation
clip_model_name = "openai/clip-vit-base-patch16"
clip_model = CLIPModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

# Function to generate image embedding
def image_embedding(image):
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = clip_model.get_image_features(**inputs)
    return outputs[0].cpu().numpy()

# Function to display image
def display_image(image_path):
    img = Image.open(image_path)
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Query Image: {os.path.basename(image_path)}')
    plt.show()

# Function to search for similar items in Qdrant
def search_similar_items(query_image, client, collection_name):
    query_embedding = image_embedding(query_image)
    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=2  # Get top 2 results
    )

    if len(search_result) < 2:
        print("Not enough results found.")
        return None, None

    return search_result[0], search_result[1]

# Function to process detection, cropping, segmentation, and search
def detect_crop_and_segment(image_path):
    results = model(image_path)
    for result in results:
        boxes = result.boxes.xyxy.cpu().detach().numpy()
        labels = result.boxes.cls.cpu().detach().numpy()
        scores = result.boxes.conf.cpu().detach().numpy()
        masks = result.masks.data.cpu().detach().numpy() if result.masks is not None else None

        threshold = 0.3
        for i, label in enumerate(labels):
            if scores[i] >= threshold:
                x1, y1, x2, y2 = map(int, boxes[i])
                im = Image.open(image_path).convert('RGB')
                im_np = np.array(im)
                im_crop = im_np[y1:y2, x1:x2]

                if masks is not None:
                    mask = masks[i]
                    mask_resized = cv2.resize(mask, (x2-x1, y2-y1), interpolation=cv2.INTER_NEAREST)
                    im_segmented = cv2.bitwise_and(im_crop, im_crop, mask=mask_resized.astype(np.uint8))
                else:
                    im_segmented = im_crop

                return Image.fromarray(im_segmented)

    print(f"No target objects detected in '{image_path}'.")
    return None

# Function to apply conditions on search results and display output
def apply_conditions_and_display(image_path):
    segmented_image = detect_crop_and_segment(image_path)
    if segmented_image is None:
        return

    top_1, top_2 = search_similar_items(segmented_image, qdrant_client, collection_name)
    if top_1 is None or top_2 is None:
        return

    score_1 = top_1.score
    score_2 = top_2.score

    # Condition 1: Score > 0.9 and difference between top 1 and top 2 is > 2%
    if score_1 > 0.9:
        if (score_1 - score_2) >= 0.02:
            print(f"Class Name: {top_1.payload.get('class', 'Unknown')}, Score: {score_1}")
        else:
            print("Top 1 and top 2 scores are close. Please retake the image.")

    # Condition 2: 0.85 < score <= 0.9
    elif 0.85 < score_1 <= 0.9:
        print("This drug might be in the class. Please retake the image.")

    # Condition 3: score <= 0.85
    else:
        print("This drug is not in the class. Saving as a new class.")
        # Add the logic for saving as a new class here

# Process all images in the input directory
if os.path.isdir(input_dir):
    for root, _, files in os.walk(input_dir):
        for file_name in sorted(files):
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                file_path = os.path.join(root, file_name)
                apply_conditions_and_display(file_path)
else:
    print(f"Invalid path: {input_dir}")

#detect and crop

In [ ]:
import os
from PIL import Image
import cv2
import numpy as np
from ultralytics import YOLO
from qdrant_client import QdrantClient
from transformers import CLIPProcessor, CLIPModel
import torch
import matplotlib.pyplot as plt

# Load your YOLOv8 model
model = YOLO('/content/drive/MyDrive/drug/best_segment.pt')

# Define the input directory
input_dir = '/content/data 50 class add_augment'

# Connect to Qdrant
qdrant_client = QdrantClient(
    url="https://8366dca9-9b40-481c-9a7c-102b62b118c2.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="Ft3t6L99uoTsJW8IOk5VH2byKU-BQYqiieuxXFBDp99wo75od0ddAw",
)
collection_name = "vector_drug"

# Load CLIPModel and processor for embedding generation
clip_model_name = "openai/clip-vit-base-patch16"
clip_model = CLIPModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

# Function to generate image embedding
def image_embedding(image):
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = clip_model.get_image_features(**inputs)
    return outputs[0].cpu().numpy()

# Function to display image
def display_image(image_path):
    img = Image.open(image_path)
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Query Image: {os.path.basename(image_path)}')
    plt.show()

# Function to search for similar items in Qdrant
def search_similar_items(query_image, client, collection_name):
    query_embedding = image_embedding(query_image)
    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=2  # Get top 2 results
    )

    if len(search_result) < 2:
        print("Not enough results found.")
        return None, None

    return search_result[0], search_result[1]

# Function to process detection and cropping
def detect_and_crop(image_path):
    results = model(image_path)
    for result in results:
        boxes = result.boxes.xyxy.cpu().detach().numpy()
        labels = result.boxes.cls.cpu().detach().numpy()
        scores = result.boxes.conf.cpu().detach().numpy()

        threshold = 0.3
        for i, label in enumerate(labels):
            if scores[i] >= threshold:
                x1, y1, x2, y2 = map(int, boxes[i])
                im = Image.open(image_path).convert('RGB')
                im_crop = im.crop((x1, y1, x2, y2))

                return im_crop

    print(f"No target objects detected in '{image_path}'.")
    return None

# Function to apply conditions on search results and display output
def apply_conditions_and_display(image_path):
    cropped_image = detect_and_crop(image_path)
    if cropped_image is None:
        return

    top_1, top_2 = search_similar_items(cropped_image, qdrant_client, collection_name)
    if top_1 is None or top_2 is None:
        return

    score_1 = top_1.score
    score_2 = top_2.score

    # Condition 1: Score > 0.9 and difference between top 1 and top 2 is > 2%
    if score_1 > 0.9:
        if (score_1 - score_2) >= 0.02:
            print(f"Class Name: {top_1.payload.get('class', 'Unknown')}, Score: {score_1}")
        else:
            print("Top 1 and top 2 scores are close. Please retake the image.")

    # Condition 2: 0.85 < score <= 0.9
    elif 0.85 < score_1 <= 0.9:
        print("This drug might be in the class. Please retake the image.")

    # Condition 3: score <= 0.85
    else:
        print("This drug is not in the class. Saving as a new class.")
        # Add the logic for saving as a new class here

# Process all images in the input directory
if os.path.isdir(input_dir):
    for root, _, files in os.walk(input_dir):
        for file_name in sorted(files):
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                file_path = os.path.join(root, file_name)
                apply_conditions_and_display(file_path)
else:
    print(f"Invalid path: {input_dir}")

#รวม

1. เอาภาพเข้ามา
2. ครอปภาพ
3. เอาภาพไปหาความเหมือนในqdrant
4. เอาผลค่าความเหมือนมาเข้าเงื่อนไข
5. แสดงผลลัพท์ภาพต้นฉบับ+ชื่อคลาสที่ทำนายได้

In [ ]:
import os
from PIL import Image
import numpy as np
from ultralytics import YOLO
from qdrant_client import QdrantClient
from transformers import CLIPProcessor, CLIPModel
import torch
import matplotlib.pyplot as plt

# Load your YOLOv8 model
model = YOLO('/content/drive/MyDrive/drug/best_new.pt')

# Define the input image path
input_image_path = '/content/Lanzaar100mg-n.jpg'

# Connect to Qdrant
qdrant_client = QdrantClient(
    url="https://8366dca9-9b40-481c-9a7c-102b62b118c2.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="Ft3t6L99uoTsJW8IOk5VH2byKU-BQYqiieuxXFBDp99wo75od0ddAw",
)
collection_name = "vector_CLIP"

# Load CLIPModel and processor for embedding generation
clip_model_name = "openai/clip-vit-large-patch14"
clip_model = CLIPModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

# Function to generate image embedding
def image_embedding(image):
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = clip_model.get_image_features(**inputs)
    return outputs[0].cpu().numpy()

# Function to search for similar items in Qdrant
def search_similar_items(query_image, client, collection_name):
    query_embedding = image_embedding(query_image)
    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=2  # Get top 2 results
    )

    if len(search_result) < 2:
        print("Not enough results found.")
        return None, None

    return search_result[0], search_result[1]

# Function to process detection and cropping
def detect_and_crop(image_path):
    results = model(image_path)
    for result in results:
        boxes = result.boxes.xyxy.cpu().detach().numpy()
        labels = result.boxes.cls.cpu().detach().numpy()
        scores = result.boxes.conf.cpu().detach().numpy()

        threshold = 0.3
        for i, label in enumerate(labels):
            if scores[i] >= threshold:
                x1, y1, x2, y2 = map(int, boxes[i])
                im = Image.open(image_path).convert('RGB')
                im_crop = im.crop((x1, y1, x2, y2))

                return im_crop

    print(f"No target objects detected in '{image_path}'.")
    return None

# Function to apply conditions, display output, and show cropped image with similarity score
def apply_conditions_and_display(image_path):
    cropped_image = detect_and_crop(image_path)
    if cropped_image is None:
        return

    top_1, top_2 = search_similar_items(cropped_image, qdrant_client, collection_name)
    if top_1 is None or top_2 is None:
        return

    score_1 = top_1.score
    score_2 = top_2.score
    class_name = top_1.payload.get('class', 'Unknown')

    # Condition 1: Score > 0.9 and difference between top 1 and top 2 is > 2%
    if score_1 > 0.9:
        if (score_1 - score_2) >= 0.02:
            prediction = f"Class Name: {class_name}, Score 1: {score_1:.4f}, Score 2: {score_2:.4f}"
        else:
            prediction = f"Top 1 and top 2 scores are close (Score 1: {score_1:.4f}, Score 2: {score_2:.4f}). Please retake the image."

    # Condition 2: 0.85 < score < 0.9
    elif 0.85 < score_1 < 0.9:
        prediction = f"This drug might be in the class. Score 1: {score_1:.4f}. Please retake the image."

    # Condition 3: score < 0.85
    else:
        prediction = f"This drug is not in the class. Score 1: {score_1:.4f}. Saving as a new class."
        # Add the logic for saving as a new class here

    # Display the cropped image with similarity score
    plt.figure(figsize=(8, 8))
    plt.imshow(cropped_image)
    plt.axis('off')
    plt.title(prediction)
    plt.show()

# Process the single image
apply_conditions_and_display(input_image_path)